In [66]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder,StandardScaler

In [67]:
df=pd.read_csv("data/stroke-data.csv")
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [68]:
#droping id column , unwanted column
df.drop('id',axis=1,inplace=True)
df.columns

Index(['gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
       'smoking_status', 'stroke'],
      dtype='object')

In [69]:
imputer = SimpleImputer(strategy = 'mean')
df['bmi']=imputer.fit_transform(df[['bmi']])
#encoded_data= df.copy()
#2nd Scale down the numerical features
#features_to_scale=['age','bmi']
#scaler = MinMaxScaler()
#encoded_data[features_to_scale]=scaler.fit_transform(encoded_data[features_to_scale])

In [70]:
#as the 'avg glucose level dosent have a normal distribution 
from sklearn.preprocessing import QuantileTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
# Initialize QuantileTransformer
#scaler = QuantileTransformer(output_distribution='uniform')

# Apply quantile transformation to avg_glucose_level
#encoded_data['avg_glucose_level'] = scaler.fit_transform(encoded_data[['avg_glucose_level']])

In [71]:
#df1 = encoded_data.copy()
#df1.columns
df.columns

Index(['gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
       'smoking_status', 'stroke'],
      dtype='object')

In [72]:
X = df.drop('stroke',axis=1)
y = df['stroke']

In [75]:
#create column transformer with 3 types of transformers
num_features = X.select_dtypes(exclude='object').columns
cat_features = X.select_dtypes(include='object').columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder",oh_transformer, cat_features),
        ("StandardScaler",numeric_transformer, num_features),
    ]
)
# Train the model
#pipeline.fit(X, y)
# Save the pipeline
#joblib.dump(pipeline, 'stroke_pipeline.pkl')

In [76]:
X= preprocessor.fit_transform(X)

In [77]:
X.shape

(5110, 21)

In [78]:
X

array([[ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00, ...,
         4.18503199e+00,  2.70637544e+00,  1.00123401e+00],
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.38946800e-01,  2.12155854e+00,  4.61555355e-16],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00, ...,
         4.18503199e+00, -5.02830130e-03,  4.68577254e-01],
       ...,
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.38946800e-01, -5.11442636e-01,  2.21736316e-01],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00, ...,
        -2.38946800e-01,  1.32825706e+00, -4.27845098e-01],
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.38946800e-01, -4.60867458e-01, -3.49895329e-01]])

In [80]:
X_train, X_test, y_train , y_test = train_test_split(X,y,test_size=0.2, random_state=42)
X_train.shape , X_test.shape

((4088, 21), (1022, 21))

In [54]:
# List of columns to one-hot encode
'''columns_to_encode = ['Residence_type', 'work_type', 'smoking_status','ever_married','gender']

# Iterate through each column and apply pd.get_dummies
for column in columns_to_encode:
    encoded_column = pd.get_dummies(df1[column], prefix=column)
    df1 = pd.concat([df1, encoded_column], axis=1)
    df1 = df1.drop(columns=[column],axis=1)

# Convert boolean to integers
df1 = df1.astype(int)'''

In [81]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay,accuracy_score

In [82]:
def evaluate_model(true,predicted):
    accuracy= accuracy_score(true,predicted)
    return accuracy

In [83]:
models = {
    "Decision Tree": DecisionTreeClassifier(criterion='entropy',splitter='best',max_depth=None),
    "Random Forest Classifier": RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=None, random_state=42),
    "XGBClassifier": xgb.XGBClassifier(objective='binary:logistic', max_depth=3, learning_rate=0.1, n_estimators=100),
    "Logistic Regression": LogisticRegression()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_accuracy = evaluate_model(y_train, y_train_pred)

    model_test_accuracy = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Accuracy Score: {:.4f}".format(model_train_accuracy))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- R2 Score: {:.4f}".format(model_test_accuracy))
    r2_list.append(model_test_accuracy)
    
    print('='*35)
    print('\n')

Decision Tree
Model performance for Training set
- Accuracy Score: 1.0000
----------------------------------
Model performance for Test set
- R2 Score: 0.9041


Random Forest Classifier
Model performance for Training set
- Accuracy Score: 1.0000
----------------------------------
Model performance for Test set
- R2 Score: 0.9393


XGBClassifier
Model performance for Training set
- Accuracy Score: 0.9552
----------------------------------
Model performance for Test set
- R2 Score: 0.9403


Logistic Regression
Model performance for Training set
- Accuracy Score: 0.9545
----------------------------------
Model performance for Test set
- R2 Score: 0.9393




In [85]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'Accuracy_Score']).sort_values(by=["Accuracy_Score"],ascending=False)

,Model Name,Accuracy_Score
2,XGBClassifier,0.940313
1,Random Forest Classifier,0.939335
3,Logistic Regression,0.939335
0,Decision Tree,0.904110


In [89]:
# Instantiate XGBClassifier
xgb_model = xgb.XGBClassifier(objective='binary:logistic', max_depth=3, learning_rate=0.1, n_estimators=100)
# Train the model
xgb_model.fit(X_train, y_train)
xgb_y_pred = xgb_model.predict(X_test)
print(f'Accuracy {accuracy_score(y_test,xgb_y_pred)}')

Accuracy 0.9403131115459883


In [91]:
import joblib
joblib.dump(xgb_model,"strokemodel.pkl")

['strokemodel.pkl']

In [90]:
#df1.rename(columns={'work_type_Self-employed':'work_type_Selfemployed',
#           'smoking_status_formerly smoked':'smoking_status_formerly_smoked',
#           'smoking_status_never smoked':'smoking_status_never_smoked'},inplace=True)
#df1.columns

In [86]:
#from sklearn.tree import DecisionTreeClassifier
#clf = DecisionTreeClassifier(criterion='entropy',splitter='best',max_depth=None)
#clf.fit(X_train,y_train)

In [87]:
#from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay,accuracy_score
#y_pred = clf.predict(X_test)
#print(f'Accuracy {accuracy_score(y_test,y_pred)}')

In [88]:
'''from sklearn.ensemble import RandomForestClassifier

# Instantiate RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=None, random_state=42)

# Train the model
rf_clf.fit(X_train, y_train)
y_pred = rf_clf.predict(X_test)
print(f'Accuracy {accuracy_score(y_test,y_pred)}')'''

"from sklearn.ensemble import RandomForestClassifier\n\n# Instantiate RandomForestClassifier\nrf_clf = RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=None, random_state=42)\n\n# Train the model\nrf_clf.fit(X_train, y_train)\ny_pred = rf_clf.predict(X_test)\nprint(f'Accuracy {accuracy_score(y_test,y_pred)}')"

In [64]:
'''from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Example data preparation (replace with your actual data)

# Instantiate Logistic Regression model
logreg = LogisticRegression()

# Train the model
logreg.fit(X_train, y_train)

# Predict on validation set
y_pred = logreg.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')'''

Accuracy: 0.9393346379647749
